In [1]:
%matplotlib ipympl
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.ndimage import gaussian_filter
from scipy.spatial.distance import cdist

from vizlib import *

In [2]:
filename = "/Volumes/DATA/amatter_24_01_2024_card_worms_only.xyzv"
filename = "/media/matt/DATA/amatter_24_01_2024_card_worms_only.xyzv"
#filename = "/media/matt/DATA/amatter_02_02_2024_worms_only.xyzv"
filename = "/media/matt/SCRATCH/ActiveMatterWorms/data_02_02_2024/amatter_02_02_2024_worms_only.xyzv"
N,nframes,pos_data,vel_data = load_data(filename)

64000  particles per frame
Loading  3001  frames of data...


: 

: 

In [ ]:
vector_field = get_particle_vecs_frame(N,80,-1,pos_data)

In [ ]:
"""plotting the vectors of each of the individual particles"""
fig,ax = plt.subplots(1,1,figsize=(10,10))
quiver = ax.quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax.set_box_aspect(1)
ax.set_adjustable("datalim")
plt.show()

In [ ]:
max_x = np.max(vector_field[:,0])
min_x = np.min(vector_field[:,0])
max_y = np.max(vector_field[:,1])
min_y = np.min(vector_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

num_interpolation_points = 40
x_grid,y_grid = np.meshgrid(np.linspace(min_x,max_x,num_interpolation_points),
                            np.linspace(min_y,max_y,num_interpolation_points))

Qxx = np.zeros((num_interpolation_points,num_interpolation_points))
Qxy = np.zeros((num_interpolation_points,num_interpolation_points))
nx = np.zeros((num_interpolation_points,num_interpolation_points))
ny = np.zeros((num_interpolation_points,num_interpolation_points))

for i in np.arange(0,x_grid.shape[0]-1):
   for j in np.arange(0,y_grid.shape[1]-1):
      # getting the cell boundary points
      mask = (vector_field[:,0] >= x_grid[i,j]) & (vector_field[:,0] < x_grid[i,j+1]) & \
             (vector_field[:,1] >= y_grid[i,j]) & (vector_field[:,1] < y_grid[i+1,j])
      # selecting particles that are in that cell
      if np.any(mask):
         x_cell_vals = vector_field[mask,3]
         y_cell_vals = vector_field[mask,4]
      else:
         x_cell_vals = 0.0
         y_cell_vals = 0.0

      Qxx[i,j] = 0.5*(3*np.mean(x_cell_vals*x_cell_vals)-1)
      Qxy[i,j] = 0.5*(3*np.mean(x_cell_vals*y_cell_vals))
np.nan_to_num(Qxx, copy=False, nan=0.0)
np.nan_to_num(Qxy, copy=False, nan=0.0)
for i in np.arange(0,x_grid.shape[0]-1):
   for j in np.arange(0,y_grid.shape[1]-1):
      #S[x,y] = 0.5*np.sqrt(self.trace_Q_squared(self.Q[x,y,0],self.Q[x,y,1]))
      a = Qxx[i,j]
      b = Qxy[i,j]
      eigenvalues,eigenvectors = np.linalg.eig(np.array([[a,b],[b,-a]]))
      nx[i,j] = eigenvectors[0,np.argmax(eigenvalues)]
      ny[i,j] = eigenvectors[1,np.argmax(eigenvalues)]

"""plotting the vectors of each of the individual particles"""
fig,ax = plt.subplots(1,1,figsize=(10,10))
quiver = ax.quiver(x_grid, #x
                   y_grid, #y
                   nx, #u
                   ny, #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax.set_box_aspect(1)
ax.set_adjustable("datalim")
plt.show()



In [ ]:
dx = x_grid[0,1] - x_grid[0,0]
dy = y_grid[1,0] - y_grid[0,0]

dq11_tensor_dx = np.zeros_like(x_grid)
dq12_tensor_dx = np.zeros_like(x_grid)
dq11_tensor_dy = np.zeros_like(y_grid)
dq12_tensor_dy = np.zeros_like(y_grid)

dq11_tensor_dx[:, 1:-1] = (Qxx[:, 2:] - Qxx[:, :-2]) / (2 * dx)
dq11_tensor_dx[:, 0] = (Qxx[:, 1] - Qxx[:, 0]) / dx
dq11_tensor_dx[:, -1] = (Qxx[:, -1] - Qxx[:, -2]) / dx

dq12_tensor_dx[:, 1:-1] = (Qxy[:, 2:] - Qxy[:, :-2]) / (2 * dx)
dq12_tensor_dx[:, 0] = (Qxy[:, 1] - Qxy[:, 0]) / dx
dq12_tensor_dx[:, -1] = (Qxy[:, -1] - Qxy[:, -2]) / dx

dq11_tensor_dy[1:-1, :] = (Qxx[2:, :] - Qxx[:-2, :]) / (2 * dy)
dq11_tensor_dy[0, :] = (Qxx[1, :] - Qxx[0, :]) / dy
dq11_tensor_dy[-1, :] = (Qxx[-1, :] - Qxx[-2, :]) / dy

dq12_tensor_dy[1:-1, :] = (Qxy[2:, :] - Qxy[:-2, :]) / (2 * dy)
dq12_tensor_dy[0, :] = (Qxy[1, :] - Qxy[0, :]) / dy
dq12_tensor_dy[-1, :] = (Qxy[-1, :] - Qxy[-2, :]) / dy

dij = dq11_tensor_dx * dq12_tensor_dy - dq11_tensor_dy*dq12_tensor_dx
print(dij.shape)

fig,ax2 = plt.subplots(1,1,figsize=(10,10))
quiver = ax2.quiver(x_grid, #x
                   y_grid, #y
                   nx, #u
                   ny, #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
pcm = ax2.imshow(dij,cmap="hot",interpolation="bicubic",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
#pcm = ax2.imshow(dij,cmap="hot",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=ax2)
ax2.set_box_aspect(1)
ax2.set_adjustable("datalim")
plt.show()

In [ ]:
""" animating the previous plot"""
from matplotlib.animation import FuncAnimation

# Assuming you have a list of vector fields called 'vector_fields'
vector_fields = []
for iframe in tqdm.tqdm(range(nframes)):
    vector_field_frame = np.zeros((N,6))
    vector_field_frame[:,0:3] = pos_data[iframe]
    vector_field_frame[:,3:6] = vel_data[iframe]
    #vector_field_frame = get_particle_vecs_frame(N,iframe,pos_data,disp=False)
    #vector_grid_frame = interp_particle_vecs(vector_field_frame,100,"cubic",disp=False)
    vector_fields.append(vector_field_frame)
    
fig, ax = plt.subplots(1, 1)

def update(frame):
   ax.clear()
   vector_field_frame = vector_fields[frame]
   max_x = np.max(vector_field_frame[:,0])
   min_x = np.min(vector_field_frame[:,0])
   max_y = np.max(vector_field_frame[:,1])
   min_y = np.min(vector_field_frame[:,1])

   num_interpolation_points = 40
   x_grid,y_grid = np.meshgrid(np.linspace(min_x,max_x,num_interpolation_points),
                              np.linspace(min_y,max_y,num_interpolation_points))

   Qxx = np.zeros((num_interpolation_points,num_interpolation_points))
   Qxy = np.zeros((num_interpolation_points,num_interpolation_points))
   nx = np.zeros((num_interpolation_points,num_interpolation_points))
   ny = np.zeros((num_interpolation_points,num_interpolation_points))

   for i in np.arange(0,x_grid.shape[0]-1):
      for j in np.arange(0,y_grid.shape[1]-1):
         # getting the cell boundary points
         mask = (vector_field_frame[:,0] >= x_grid[i,j]) & (vector_field_frame[:,0] < x_grid[i,j+1]) & \
               (vector_field_frame[:,1] >= y_grid[i,j]) & (vector_field_frame[:,1] < y_grid[i+1,j])
         # selecting particles that are in that cell
         if np.any(mask):
            x_cell_vals = vector_field_frame[mask,3]
            y_cell_vals = vector_field_frame[mask,4]
         else:
            x_cell_vals = 0.0
            y_cell_vals = 0.0

         Qxx[i,j] = 0.5*(3*np.mean(x_cell_vals*x_cell_vals)-1)
         Qxy[i,j] = 0.5*(3*np.mean(x_cell_vals*y_cell_vals))
   np.nan_to_num(Qxx, copy=False, nan=0.0)
   np.nan_to_num(Qxy, copy=False, nan=0.0)
   for i in np.arange(0,x_grid.shape[0]-1):
      for j in np.arange(0,y_grid.shape[1]-1):
         #S[x,y] = 0.5*np.sqrt(self.trace_Q_squared(self.Q[x,y,0],self.Q[x,y,1]))
         a = Qxx[i,j]
         b = Qxy[i,j]
         eigenvalues,eigenvectors = np.linalg.eig(np.array([[a,b],[b,-a]]))
         nx[i,j] = eigenvectors[0,np.argmax(eigenvalues)]
         ny[i,j] = eigenvectors[1,np.argmax(eigenvalues)]

   """plotting the vectors of each of the individual particles"""
   dx = x_grid[0,1] - x_grid[0,0]
   dy = y_grid[1,0] - y_grid[0,0]

   dq11_tensor_dx = np.zeros_like(x_grid)
   dq12_tensor_dx = np.zeros_like(x_grid)
   dq11_tensor_dy = np.zeros_like(y_grid)
   dq12_tensor_dy = np.zeros_like(y_grid)

   dq11_tensor_dx[:, 1:-1] = (Qxx[:, 2:] - Qxx[:, :-2]) / (2 * dx)
   dq11_tensor_dx[:, 0] = (Qxx[:, 1] - Qxx[:, 0]) / dx
   dq11_tensor_dx[:, -1] = (Qxx[:, -1] - Qxx[:, -2]) / dx

   dq12_tensor_dx[:, 1:-1] = (Qxy[:, 2:] - Qxy[:, :-2]) / (2 * dx)
   dq12_tensor_dx[:, 0] = (Qxy[:, 1] - Qxy[:, 0]) / dx
   dq12_tensor_dx[:, -1] = (Qxy[:, -1] - Qxy[:, -2]) / dx

   dq11_tensor_dy[1:-1, :] = (Qxx[2:, :] - Qxx[:-2, :]) / (2 * dy)
   dq11_tensor_dy[0, :] = (Qxx[1, :] - Qxx[0, :]) / dy
   dq11_tensor_dy[-1, :] = (Qxx[-1, :] - Qxx[-2, :]) / dy

   dq12_tensor_dy[1:-1, :] = (Qxy[2:, :] - Qxy[:-2, :]) / (2 * dy)
   dq12_tensor_dy[0, :] = (Qxy[1, :] - Qxy[0, :]) / dy
   dq12_tensor_dy[-1, :] = (Qxy[-1, :] - Qxy[-2, :]) / dy

   dij = dq11_tensor_dx * dq12_tensor_dy - dq11_tensor_dy*dq12_tensor_dx

   quiver = ax.quiver(x_grid, #x
                     y_grid, #y
                     nx, #u
                     ny, #v
                     pivot='mid',
                     headlength=0,
                     headwidth=0,
                     headaxislength=0,
                     color="tab:blue",
                     scale_units='xy',
                     scale=0.25
   )
   pcm = ax.imshow(dij,cmap="hot",interpolation="bicubic",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
   #pcm = ax2.imshow(dij,cmap="hot",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
   fig.colorbar(pcm, ax=ax)
   ax.set_box_aspect(1)
   ax.set_adjustable("datalim")
   ax.set_title(f'Timestep {frame}')
    
# Create the animation
animation = FuncAnimation(fig, update, frames=nframes, interval=100, repeat=False)
#saving the animation
animation.save('vector_field_animation.mp4', writer='ffmpeg', fps=25)
#plt.show()

In [ ]:
max_x = np.max(vector_field[:,0])
min_x = np.min(vector_field[:,0])
max_y = np.max(vector_field[:,1])
min_y = np.min(vector_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)
num_interpolation_points = 100
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))
# Interpolate the x and y components separately using bilinear interpolation
grid_u = interpolate.griddata((vector_field[:,0], vector_field[:,1]), vector_field[:,3], (grid_x, grid_y), method='linear')
grid_v = interpolate.griddata((vector_field[:,0], vector_field[:,1]), vector_field[:,4], (grid_x, grid_y), method='linear')

Qxx = np.zeros((num_interpolation_points,num_interpolation_points))
Qxy = np.zeros((num_interpolation_points,num_interpolation_points))
nx = np.zeros((num_interpolation_points,num_interpolation_points))
ny = np.zeros((num_interpolation_points,num_interpolation_points))
for i in range(num_interpolation_points):
   for j in range(num_interpolation_points):
      Qxx[i,j] = 0.5*(3*grid_u[i,j]*grid_u[i,j]-1)
      Qxy[i,j] = 0.5*(3*grid_u[i,j]*grid_v[i,j])
      #S[x,y] = 0.5*np.sqrt(self.trace_Q_squared(self.Q[x,y,0],self.Q[x,y,1]))
      a = Qxx[i,j]
      b = Qxy[i,j]
      if not (np.isnan(a) or np.isnan(b)):
         eigenvalues,eigenvectors = np.linalg.eig(np.array([[a,b],[b,-a]]))
         nx[i,j] = eigenvectors[0,np.argmax(eigenvalues)]
         ny[i,j] = eigenvectors[1,np.argmax(eigenvalues)]

"""plotting the vectors of each of the individual particles"""
fig,ax = plt.subplots(1,1,figsize=(10,10))
quiver = ax.quiver(grid_x, #x
                   grid_y, #y
                   nx, #u
                   ny, #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax.set_box_aspect(1)
ax.set_adjustable("datalim")
plt.show()

In [ ]:
dx = x_grid[0,1] - x_grid[0,0]
dy = y_grid[1,0] - y_grid[0,0]

dq11_tensor_dx = np.zeros_like(x_grid)
dq12_tensor_dx = np.zeros_like(x_grid)
dq11_tensor_dy = np.zeros_like(y_grid)
dq12_tensor_dy = np.zeros_like(y_grid)

dq11_tensor_dx[:, 1:-1] = (Qxx[:, 2:] - Qxx[:, :-2]) / (2 * dx)
dq11_tensor_dx[:, 0] = (Qxx[:, 1] - Qxx[:, 0]) / dx
dq11_tensor_dx[:, -1] = (Qxx[:, -1] - Qxx[:, -2]) / dx

dq12_tensor_dx[:, 1:-1] = (Qxy[:, 2:] - Qxy[:, :-2]) / (2 * dx)
dq12_tensor_dx[:, 0] = (Qxy[:, 1] - Qxy[:, 0]) / dx
dq12_tensor_dx[:, -1] = (Qxy[:, -1] - Qxy[:, -2]) / dx

dq11_tensor_dy[1:-1, :] = (Qxx[2:, :] - Qxx[:-2, :]) / (2 * dy)
dq11_tensor_dy[0, :] = (Qxx[1, :] - Qxx[0, :]) / dy
dq11_tensor_dy[-1, :] = (Qxx[-1, :] - Qxx[-2, :]) / dy

dq12_tensor_dy[1:-1, :] = (Qxy[2:, :] - Qxy[:-2, :]) / (2 * dy)
dq12_tensor_dy[0, :] = (Qxy[1, :] - Qxy[0, :]) / dy
dq12_tensor_dy[-1, :] = (Qxy[-1, :] - Qxy[-2, :]) / dy

dij = dq11_tensor_dx * dq12_tensor_dy - dq11_tensor_dy*dq12_tensor_dx
print(dij.shape)

fig,ax2 = plt.subplots(1,1,figsize=(10,10))
quiver = ax2.quiver(x_grid, #x
                   y_grid, #y
                   nx, #u
                   ny, #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
pcm = ax2.imshow(dij,cmap="hot",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=ax2)
ax2.set_box_aspect(1)
ax2.set_adjustable("datalim")
plt.show()